# Calibration Procedure

* Compute center offset:
 - Set $\lambda_{\rm center}$ to set of known spectral lines
 - Measure pixel position of each: 
 - average each to determine central pixel $n_o$
 
|   $\lambda_{\rm center}$ | Pixel |           
| ----------------------:  |:------:|
| 0   nm                   | 508 | 
| 445 nm                   | 6.22      |  
| 901 nm                   | 3.1      | 


 
* Compute spectrometer calibration angles/length ($\ f_L, \delta, \gamma$)
 * Move known spectral line $\lambda_o$ to left and right sides of detector
 * record $\lambda_{\rm center}$ and pixel position for each 
 * Compute best fit of $\ f_{\rm calib}$


| $\lambda_o$   | Side | $\lambda_{\rm center}$| Pixel  |
| ------------- | ---- |:----------------------|-------:|
| 809.4 nm      | R    |729.4910 nm            |508     |
| 809.4 nm      | L    |899.5830 nm            |  4     |
| ...           | ...  | ...                   |...     |



# Optimization Function

Optimize for 3 parameters:
 * $f_L$: Focal length of spectrometer
 * $\delta$: Detector angle (The angle of the image plane relative to the plane perpendicular to the spectrograph focal axis at the center of the image plane)
 * $\gamma$: inclusion angle

from experiment:
 * $n =  n_{px} - n_o$: Pixel from central pixel
 * $\lambda_{\rm center}$: Wavelength of center pixel 
 * $\lambda_p$: Wavelength of pixel $n$
 
Fixed Constants:
 * $m$: Diffraction order (typically one)
 * $x_{\rm pixel}$: pixel size
 * $d_{grating}$: Grating pitch (1/(groves / mm))
    
residual: (wl,  wl_p, n, f, delta,gamma)

We measure pixel position ($n$) of a known wavelength ($\lambda_p$) for multple peaks and spectrometer positions and find the best fit parameters $\ f_L, \delta, \gamma$:

$$ \lambda_p = f_{\rm calib} ( n,  \lambda_{\rm center}, 
    \underbrace{m, x_{\rm pixel}, d_{\rm grating}}_{\rm spec\ params}, 
    \overbrace{f_L,\ \ \delta,\ \ \gamma}^{\rm Calibration\ params} ) $$

$$ \lambda_p = \frac{d}{m} \cdot \left[ \sin( \psi - \frac{\gamma}{2}) + \sin(\psi+\frac{\gamma}{2} + \eta) \right]$$

Where

$$ \psi = \arcsin \left[ \frac{ m\ \lambda_{\rm center} } { 2\ d_{\rm grating} \cos(\frac{\gamma}{2})} \right] $$

$$ \eta = \arctan \left[ \frac{ n\ x_{pixel} \cos{\delta}} {f_L + n\ x_{pixel} \sin(\delta)} \right]$$

$$n =  n_{px} - n_o$$



In [ ]:
from __future__ import division
import numpy as np
import  matplotlib.pyplot as plt
from pprint import pprint
import numpy as np

%matplotlib notebook

In [ ]:
def wl_p_calib_p2(px, n0, offset_adjust, wl_center, m_order, d_grating, x_pixel, f, delta, gamma, curvature):
    #consts
    #d_grating = 1./150. #mm
    #x_pixel   = 16e-3 # mm
    #m_order   = 1 # diffraction order, unitless
    n = px - (n0+offset_adjust*wl_center)

    psi = np.arcsin( m_order* wl_center / (2*d_grating*np.cos(gamma/2.)))
    eta = np.arctan(n*x_pixel*np.cos(delta) / (f+n*x_pixel*np.sin(delta)))
    
    return ((d_grating/m_order)
                    *(np.sin(psi-0.5*gamma)
                      + np.sin(psi+0.5*gamma+eta))) + curvature*n**2

In [ ]:
from scipy.optimize import least_squares

def fit_residual(
                # optimization parameters
                opt_params, #  (f, delta, gamma, curvature)
                # other params and data
                px, n0, offset_adjust, wl_center, m_order, d_grating, x_pixel,
                wl_actual
                ):
    
    (f, delta, gamma, curvature) = opt_params
    wl_model = wl_p_calib_p2(px, n0, offset_adjust, wl_center, m_order, d_grating, x_pixel, f, delta, gamma,curvature)
    return wl_model - wl_actual

# grating 1 (600 g/mm Bz 300)

In [ ]:
# grating 1 (600 g/mm Bz 300)

wl_center_data = np.array([
[365.02, 508],
[404.66, 508],
[435.83, 508],
[546.07, 509],
[1013.98, 507],
])
    
n0 = np.mean(wl_center_data[:,1])
n0    
plt.figure()
plt.plot(wl_center_data[:,0], wl_center_data[:,1])
plt.axhline(n0)

In [ ]:
dispersion_data = np.array([
#wl_actual, wl_center, pixel
[365.02, 265.02, 905],
[404.66, 304.66, 909],
[435.83, 335.83, 913],
[546.07, 446.07, 923],
[1013.98, 913.98, 988],
[365.02, 365.02, 508],
[404.66, 404.66, 508],
[435.83, 435.83, 508],
[546.07, 546.07, 509],
[1013.98, 1013.98, 507],
[365.02, 465.02, 103],
[404.66, 504.66, 100],
[435.83, 535.83, 97],
[546.07, 646.07, 85],
[1013.98, 1113.98, 23],
])


In [ ]:
# MAKE SURE THE INITIAL PARAMETERS ARE GOOD !!! CHECK RESIDUAL FOR THE GOOD FITTING and adapt manually if required.
initial_guess = (150e6,0.9,0.5,0)

kwargs = dict(
    px=dispersion_data[:,2], 
    n0=np.mean(wl_center_data[:,1]),
    wl_center=dispersion_data[:,1], # nm
    m_order=1,
    d_grating=1/150.*1e6, # nm
    x_pixel=26e3, #nm
    wl_actual=dispersion_data[:,0], # nm
    offset_adjust = 0
)
result = least_squares(fit_residual, initial_guess, kwargs=kwargs)
result.x

In [ ]:
kwargs = dict(
    px=dispersion_data[:,2], 
    #px=wl_center_data[:,1],
    n0=np.mean(wl_center_data[:,1]),
    wl_center=dispersion_data[:,1],
    #wl_center=wl_center_data[:,0]*1e-6,
    m_order=1,
    d_grating=1/150.*1e6, # nm
    x_pixel=26e3, #nm
    #wl_actual=dispersion_data[:,0]*1e-6,
    f = result.x[0],
    delta = result.x[1],
    gamma = result.x[2],
    offset_adjust = 0,
    curvature = result.x[3]
)

wl_p_calib_p2(**kwargs) - dispersion_data[:,0]

In [ ]:
# to store for ini file
# f, delta, gamma, n0, offset_adjust, d_grating, x_pixel, curvature
Y = 'f, delta, gamma, n0, offset_adjust, d_grating, x_pixel, curvature'.split(', ')
str([ kwargs[x] for x in Y ])
#kwargs

In [ ]:
fig = plt.figure()
f, delta, gamma, n0, offset_adjust, d_grating, x_pixel, curvature = [np.float64(330605663.74965495), np.float64(-0.20488367116307532), np.float64(2.021864300924973), np.float64(508.0), 0, 6666.666666666667, 26000.0, np.float64(3.1224154313329654e-06)]
residual = wl_p_calib_p2(dispersion_data[:,2], n0, offset_adjust, dispersion_data[:,1], 1, d_grating, x_pixel, f, delta, gamma, curvature)-dispersion_data[:,0]

#plt.plot(wl_p_calib_p2(**kwargs))
plt.plot(residual,'.')
plt.title("Fit residuals")
plt.show()